<p align="center"><img src="../Additionals/Empa-Workshops-Template-Banner2.png" alt="University Workshops" style="display: block; margin: 0 auto" height=/></p>

## Accelerator Workshops'a Hoşgeldiniz!

Empa Electronics tarafından düzenlenen Accelerators Workshops etkinlikleri serimizin Uçta Yapay Zeka adımına hoşgeldiniz. Bu açık-kaynak repository, workshop etkiliğimizde deneyimleyeceğiniz "ST Platformları İçin Edge-AI Çözümleri Geliştirme" uygulaması çalışma ortamını edinebilmeniz ve aktivitelere kolaylıkla eşlik edebilmeniz için sizinle paylaşılmıştır.

Bu script, uygulama içeriğinde geliştirilen modelin testi için oluşturulmuştur.
  

**Uygulama Adımları:**

1. Gereksinimlerin Dahil Edilmesi

2. Eğitilmiş Modelin Yüklenmesi

3. Seri Port ile Veri Örneği Okuma & Eğitilmiş Modeli Kullanarak Tahminleme

## 1. Gereksinimlerin Dahil Edilmesi

Gerekli modeller için import işlemi:

In [10]:

import tensorflow as tf
import serial
import numpy as np

## 2. Eğitilmiş Modelin Yüklenmesi

Loading the trained model:

In [11]:
model = tf.keras.models.load_model("Models/Model_CNN_Hand_Character_Recognition_0_9192.h5")

Defining the class mapping dictionary:

In [12]:
# class ID to class name mapping
classes = {0: "CIRCLE",
           1: "HORIZONTAL",
           2: "STANDBY",
           3: "TRIANGLE",
           4: "VERTICAL"}

## 3. Seri Port ile Veri Örneği Okuma & Eğitilmiş Modeli Kullanarak Tahminleme

Seri port konfigürasyonu için gerekli değerlerin tanımlanması:

In [13]:
serial_port = '/dev/ttyACM0'
baud_rate = 115200
time_out = 2

Seri port üzerinden veri okuma & okunan veri örneklerinin tahminlenmesi

In [ ]:
# collect data, get inference from model
while True:
    # open the serial port, set baudrate, set timeout
    with serial.Serial(serial_port, baud_rate, timeout=time_out) as ser_read:
        # read the line from serial port
        x = ser_read.readline()
        # parse the line into numpy array
        line = np.array(str(x).replace("b'", "").replace("\\n'", "").replace("\\r", "").split(" ")[:-1])
        # ignore the line if the length is not 6*128=768
        if len(line) == 768:
            # convert the array of strings to array of floating point numbers
            line = line.astype(np.float32)
            # reshape the array for model
            line = line.reshape(128,6)
            # add a batch shape to the array
            line = np.expand_dims(line, axis=0)
            # get the model output
            out = model(line)
            # model outputs a tensor, convert it to numpy array
            out = np.array(out)
            # get the highest scoring output as predicted class
            class_index = np.argmax(out)
            # get the class name from dict
            class_name = classes[class_index]
            # print the results
            print(f"\rProbabilities -> CIRCLE: {out[0,0]:.2f} HORIZONTAL: {out[0,1]:.2f} STANDBY: {out[0,2]:.2f} TRIANGLE: {out[0,3]:.2f} VERTICAL: {out[0,4]:.2f} --- Prediction: {class_name} {' '*50}",end="")